In [ ]:
# default_exp learner

# Learner

> Learner class and traing loop.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export 
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import os
from time import time
from deepflash2.models import Unet2D
from deepflash2.callbacks import CyclicLR
from deepflash2.lrfinder import LRFinder
from deepflash2 import metrics

In [ ]:
#export 
class Unet_Learner:
    """
    Class used to train Keras models.

    model (Model): tf.keras.Model
    train_data (Iterator): Iterator instance for training set
    val_data (Iterator):   Iterator instance for validation set
    """

    def __init__(self, train_generator, val_generator=None, model = None,
                 opt=None, loss=None, metric=None, lr=1e-5, wd=0., 
                 checkpoint_dir = 'checkpoints',
                 workers=1, name='U-Net', use_multiprocessing=False):

        if model is not None:
            if not isinstance(model, tf.keras.Model):
                raise ValueError('model must be of instance tf.keras.Model')
            self.model = model
        else:
            self.model = Unet2D()

        self.train_generator = train_generator
        self.val_generator = val_generator
        self.workers = workers
        self.use_multiprocessing = use_multiprocessing
        self.name = name
        self.checkpoint_dir = checkpoint_dir
        if not os.path.isdir(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        self.history = None

        if loss is not None:
            self.loss = loss
        else:
            self.loss = {'conv_u0d-score': weighted_softmax_cross_entropy,
                         'softmax': zero_loss}
        if metric is not None:
            self.metric = metric
        else:
            self.metric = {'softmax':
                          [tf.keras.metrics.Recall(class_id=1),
                          tf.keras.metrics.Precision(class_id=1),
                          metrics.MeanIoU2(num_classes=2, class_id=1, name='IoU')
                          ]}

        if opt is not None:
            self.opt = opt
        else:
            if wd > 0.:
                self.opt = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
            else:
                self.opt = tf.keras.optimizers.Adam(learning_rate=lr)

        # Compile model
        self._compile()

    def _compile(self):
        self.model.compile(optimizer=self.opt, loss=self.loss, metrics=self.metric)
        return
    
    def save(self, name='latest'):
        model_path = os.path.join(self.checkpoint_dir, self.name + '_' + name + '.h5')
        self.model.save_weights(model_path)
        print('Model weights saved:', model_path)
    
    def load(self, name='latest', full_path=None):
        if full_path is not None:
            model_path = full_path
        else:
            model_path = os.path.join(self.checkpoint_dir, self.name + '_' + name + '.h5')
        self.model.load_weights(model_path)
        print('Model weights loaded from', model_path)

    def fit(self, final_epoch, max_lr,
                          initial_epoch=0,
                          verbose = True,
                          snapshot_interval=1,
                          snapshot_dir= 'checkpoints',
                          snapshot_prefix=None,
                          log_dir = 'logs'):
        """
        Standard model training. This method can be used with any optimizer.
        Args:
            max_lr (float): (maximum) learning rate.
                       It is recommended that you estimate lr yourself by
                       running lr_finder (and lr_plot) and visually inspect plot
                       for dramatic loss drop.
            epochs (int): Number of epochs.  Number of epochs
            checkpoint_folder (string): Folder path in which to save the model weights
                                        for each epoch.
                                        File name will be of the form:
                                        weights-{epoch:02d}-{val_loss:.2f}.hdf5
            verbose (bool):  verbose mode
        """

        callbacks = []

        # Tensorboard Callback
        tb = tf.keras.callbacks.TensorBoard(log_dir= log_dir + "/{}-{}".format(self.name, time()))
        callbacks.append(tb)

        # ModelCheckpoint Callback
        if snapshot_prefix is not None:
            if not os.path.isdir(snapshot_dir):
                os.makedirs(snapshot_dir)
            c_path = os.path.join(
                snapshot_dir, (snapshot_prefix if snapshot_prefix is not None else self.name))
            cp = tf.keras.callbacks.ModelCheckpoint(c_path + ".{epoch:04d}.h5", mode='auto', period=snapshot_interval)
            callbacks.append()


        hist = self.model.fit(self.train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              initial_epoch = initial_epoch,
                              validation_data = self.val_generator,
                              verbose=verbose,
                              callbacks=callbacks)

        return hist

    def fit_one_cycle(self, final_epoch, max_lr,
                      initial_epoch=0,
                      cycle_momentum = True,
                      verbose = True,
                      validation_freq=1,
                      snapshot_dir = None,
                      log_dir = 'logs'):
        """
        Train model using a version of Leslie Smith's 1cycle policy.
        This method can be used with any optimizer.
        Args:
            max_lr (float): (maximum) learning rate.
                       It is recommended that you estimate lr yourself by
                       running lr_finder (and lr_plot) and visually inspect plot
                       for dramatic loss drop.
            epochs (int): Number of epochs.  Number of epochs
            checkpoint_folder (string): Folder path in which to save the model weights
                                        for each epoch.
                                        File name will be of the form:
                                        weights-{epoch:02d}-{val_loss:.2f}.hdf5
            cycle_momentum (bool):    If True and optimizer is Adam, Nadam, or Adamax, momentum of
                                      optimzer will be cycled between 0.95 and 0.85 as described in
                                      https://arxiv.org/abs/1803.09820.
                                      Only takes effect if Adam, Nadam, or Adamax optimizer is used.
            verbose (bool):  verbose mode
        """

        callbacks = []

        # Tensorboard Callback
        tb = tf.keras.callbacks.TensorBoard(log_dir= log_dir + "/{}-{}".format(self.name, time()))
        callbacks.append(tb)

        # ModelCheckpoint Callback
        if snapshot_dir is not None:
            os.makedirs(snapshot_dir, exist_ok=True)
        else:
            snapshot_dir = self.checkpoint_dir
        c_path = os.path.join(snapshot_dir, self.name)
        #cp = tf.keras.callbacks.ModelCheckpoint(c_path + ".{epoch:04d}.h5", mode='auto', save_freq='epoch')
        cp = tf.keras.callbacks.ModelCheckpoint(c_path + "_{}_{}.h5".format(final_epoch, max_lr),
                                                mode='max', save_freq='epoch', monitor = 'val_softmax_IoU', 
                                                save_best_only=True, verbose=0)
        callbacks.append(cp)

        # CLR Callback
        if cycle_momentum:
            max_momentum = 0.95
            min_momentum = 0.85
        else:
            max_momentum = None
            min_momentum = None

        #num_samples = len(train_generator)*train_generator.batch_size
        #steps_per_epoch = np.ceil(num_samples/train_generator.batch_size)
        steps_per_epoch = len(self.train_generator)
        epochs = final_epoch-initial_epoch
        # clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
        clr = CyclicLR(base_lr=max_lr/10, max_lr=max_lr,
                                  step_size=np.ceil((steps_per_epoch*epochs)/2), # Authors suggest setting step_size = (2-8) x (training iterations in epoch)
                                  reduce_on_plateau=0,
                                  max_momentum=max_momentum,
                                  min_momentum=min_momentum,
                                  verbose=verbose)

        callbacks.append(clr)

        hist = self.model.fit(self.train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              initial_epoch = initial_epoch,
                              validation_data = self.val_generator,
                              validation_freq=validation_freq,
                              verbose=verbose,
                              callbacks=callbacks)

        hist.history['lr'] = clr.history['lr']
        hist.history['iterations'] = clr.history['iterations']
        if cycle_momentum:
            hist.history['momentum'] = clr.history['momentum']
        self.history = hist

        return hist

    def predict(self, tile_generator):
        """
        Generates output predictions for the input samples.
        Args:
            tile_generator(): number of layers to freeze
        Returns:
            softmax_scores(list)
            binary_segmentations(list)
        """

        logits, softmax_score = self.model.predict(tile_generator, verbose=1)

        smxcores = []
        segmentations = []

        for idx in range(len(tile_generator)):
            outIdx = tile_generator.image_indices[idx]
            outShape = tile_generator.image_shapes[idx]
            outSlice = tile_generator.out_slices[idx]
            inSlice = tile_generator.in_slices[idx]
            if len(smxcores) < outIdx + 1:
                smxcores.append(np.empty((*outShape, self.model.n_classes)))
                segmentations.append(np.empty(outShape))
            smxcores[outIdx][outSlice] = softmax_score[idx][inSlice]
            segmentations[outIdx][outSlice] = np.argmax(softmax_score[idx], axis=-1)[inSlice]

        return smxcores, segmentations


    def freeze(self, freeze_range=None):
        """
        If freeze_range is None, makes all layers trainable=False except last layer.
        If freeze_range is given, freezes the first <freeze_range> layers and
        unfrezes all remaining layers.
        Args:
            freeze_range(int): number of layers to freeze
        Returns:
            None
        """

        if freeze_range is None:
            # freeze everything except last layer
            for layer in self.trainModel.layers:
                if layer.name != 'conv_u0d-score':
                    #print(layer.name)
                    layer.trainable=False

        else:
            # freeze all layers up to and including layer_id
            if type(freeze_range) != type(1) or freeze_range <1:
                raise ValueError('freeze_range must be integer > 0')
            for i, layer in enumerate(self.trainModel.layers):
                if i < freeze_range:
                    layer.trainable=False
                else:
                    layer.trainable=True
        self._compile()
        return

    # https://github.com/amaiya/ktrain/blob/55e00d850b8dc43adcca64d0bb4f3f00c283c907/ktrain/core.py#L262
    def unfreeze(self, exclude_range=None):
        """
        Make every layer trainable except those in exclude_range.
        unfreeze is simply a proxy method to freeze.
        """
        # make all layers trainable
        for i, layer in enumerate(self.trainModel.layers):
            layer.trainable = True
        if exclude_range:
            for i, layer in enumerate(self.trainModel.layers[:exclude_range]):
                layer.trainable = False
        self._compile()
        return
    
    def lr_find(self, start_lr=1e-07, lr_mult=1.2, verbose=0):
       
        # Instantiate Learning Rate Finder
        self.lr_finder = LRFinder(self.model, stop_factor=4)
        # Find LR
        self.lr_finder.find(self.train_generator, start_lr=start_lr, lr_mult=lr_mult, verbose=verbose)
        
        
    def plot_loss(self):
        self.lr_finder.plot_loss()

In [ ]:
#export 
def weighted_softmax_cross_entropy(target_y, predicted_y):
    target_y, w = tf.split(target_y, num_or_size_splits=[-1, 1], axis=-1)
    w = w[...,0]
    return tf.compat.v1.losses.softmax_cross_entropy(onehot_labels = target_y,
                                                     logits = predicted_y,
                                                     weights=w,
                                                     reduction=tf.compat.v1.losses.Reduction.MEAN)

In [ ]:
#export
def zero_loss(predicted_y, target_y):
    return tf.constant([0.])